# SQL query from table names - Continued

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [3]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [13]:
context = [ {'role':'system', 'content':"""
ou are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.\

Here is the databases:\

CREATE TABLE employees (
    ID_usr INT PRIMARY KEY,
    name VARCHAR(255)
);

CREATE TABLE salary (
    ID_usr INT,
    year DATE,
    salary FLOAT,
    PRIMARY KEY (ID_usr, year)
);

CREATE TABLE studies (
    ID INT PRIMARY KEY,
    ID_usr INT,
    educational_level INT,
    Institution VARCHAR(255),
    Years DATE,
    Speciality VARCHAR(255)
);

"""} ]



In [15]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""

-- Example queries and responses:

-- 1. Find the name and salary of employees who earned more than 55000 in 2022.

-- This is your SQL:
-- SELECT e.name, s.salary
-- FROM employees e
-- JOIN salary s ON e.ID_usr = s.ID_usr
-- WHERE s.year = '2022-01-01' AND s.salary > 55000;

-- Explanation: This query retrieves the names and salaries of employees whose salary exceeds 55000 for the year 2022 by joining the employees and salary tables.

-- 2. List all institutions and specialities for employees with educational levels greater than 15.

-- This is your SQL:
-- SELECT e.name, st.Institution, st.Speciality
-- FROM employees e
-- JOIN studies st ON e.ID_usr = st.ID_usr
-- WHERE st.educational_level > 15;

-- Explanation: This query lists the names of employees alongside their institution and speciality for those with an educational level above 15. It joins the employees and studies tables.

-- 3. Count how many employees have completed studies at the 'MIT'.

-- This is your SQL:
-- SELECT COUNT(*) AS employee_count
-- FROM studies
-- WHERE Institution = 'MIT';

-- Explanation: This query counts the number of employees who studied at 'MIT' by filtering the studies table for the specified institution.
 
"""
})

In [17]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [21]:
#new
context_user = context.copy()
print(return_CCRMSQL("""The name of the employee who was the best paid""", context_user))

This is your SQL:
```sql
SELECT e.name
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
ORDER BY s.salary DESC
LIMIT 1;
```

Explanation: This query retrieves the name of the employee who was the best paid by joining the employees and salary tables, ordering the results by salary in descending order, and selecting only the top result.


In [25]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("The name of the employee who was the best paid", old_context_user))

This is your SQL:
```sql
SELECT e.name
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
ORDER BY s.salary DESC
LIMIT 1;
```

This SQL query retrieves the name of the employee who was the best paid by joining the "employees" table with the "salary" table on the ID_usr field. It then orders the result by salary in descending order and limits the output to only one row, which corresponds to the employee with the highest salary.


In [43]:
#new
print(return_CCRMSQL("The name of the institution that has the highest average salary for its employees?", context_user))

This is your SQL:
```sql
SELECT st.Institution
FROM studies st
JOIN salary s ON st.ID_usr = s.ID_usr
GROUP BY st.Institution
ORDER BY AVG(s.salary) DESC
LIMIT 1;
```

Explanation: This query calculates the average salary for employees of each institution by joining the studies and salary tables, then it orders the results in descending order of average salary and selects the institution with the highest average salary.


In [45]:
#old
print(return_CCRMSQL("The name of the institution that has the highest average salary for its employees?", old_context_user))

This is your SQL:
```sql
SELECT s.Institution, AVG(sa.salary) AS avg_salary
FROM employees e
JOIN salary sa ON e.ID_usr = sa.ID_usr
JOIN studies s ON e.ID_usr = s.ID_usr
GROUP BY s.Institution
ORDER BY avg_salary DESC
LIMIT 1;
```

This SQL query joins the three tables (employees, salary, and studies) based on the employee ID. It calculates the average salary for each institution and then selects the institution with the highest average salary.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

In [51]:
#version 1

print(return_CCRMSQL("List all institutions along with their specialties where employees have more than 5 years of experience.", context_user))

This is your SQL:
```sql
SELECT st.Institution, st.Speciality
FROM studies st
WHERE st.Years > '5 years';
```

Explanation: This query retrieves the institutions and specialties where employees have more than 5 years of experience by filtering the studies table based on the years of experience.


In [59]:
#version 2

print(return_CCRMSQL("How many employees have a salary greater than 60,000?", context_user))

This is your SQL:
```sql
SELECT COUNT(*) AS employee_count
FROM salary
WHERE salary > 60000;
```

Explanation: This query counts the number of employees whose salary is greater than 60,000 by filtering the salary table for salaries exceeding 60,000.


In [63]:
#version 3

print(return_CCRMSQL("Find the name and salary of the top 10 employees who earned more than 66000 in 2025.", context_user))

-- This is your SQL:
-- SELECT e.name, s.salary
-- FROM employees e
-- JOIN salary s ON e.ID_usr = s.ID_usr
-- WHERE s.year = '2025-01-01' AND s.salary > 66000
-- ORDER BY s.salary DESC
-- LIMIT 10;

-- Explanation: This query retrieves the names and salaries of the top 10 employees who earned more than 66000 in the year 2025. It joins the employees and salary tables, filters by salary and year, orders the results by salary in descending order, and limits the output to 10 rows.
